Initialise Global Dataset

In [1]:
import pandas as pd

Divide Dataset into Compound Families

Triphenylamine, Porphyrin, Phenothiazine, Indoline, Coumarin, Carbazole, Diphenylamine

Divide Family Sets into Training (0.7) and Test (0.3) Sets

Triphenylamine + Porphyrin --> Kennard-Stone Algorithm

 Phenothiazine, Indoline, Coumarin, Carbazole + Diphenylamine --> Modified k-medoid

Eliminate Intercorrelated Descriptors (Degree of Correlation? Which one was removed?)

Multistage Stepwise Regression to remove less important features (define important?)

Best Features Selection --> 'Best Subset selection v2.1 software' --> Subsets are selected by their ability to build MLR models

13 Triphenylamine, 12 Porphyrin, 14 Phenothiazine, 13 Indoline, 5 Coumarin, 11 Carbazole, 4 Diphenylamine

Generate multiple MLR models with number of features listed above for each family

Measure MAE of of each MLR model for each family and select the 5 best MLR models 

PLS Applied to Descriptors for 5 Best Models for each Family (improve  predictivity and robustness)

PLS Models were then used as the final predictor models

Performance Metrics

Internal --> R2, Ra2 (Explained Variance), LOOCV Correlation Coefficient (Q2loo), Variance Ratio (F)

External --> R2pred/Q2F1, Q2F2, r2m, MAE